In [ ]:
!pip3 install torch datasets transformers peft bitsandbytes

^C


  Using cached torch-2.9.1-cp313-cp313-win_amd64.whl.metadata (30 kB)
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.12.0-py3-none-any.whl.metadata (10 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached numpy-2.3.5-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached pyarrow-22.0.0-cp313-cp313-win_amd64.whl.metadata (3.3 kB)
  Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.6.0-cp313-cp313-win_amd64.whl.metadata (13 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.me

In [2]:
import torch

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments,Trainer

from peft import LoraConfig, get_peft_model, TaskType

In [3]:
model_name = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype= torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map='auto',
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name , trust_remote_code=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [4]:
lora_config = LoraConfig(
    r=16,
    lora_alpha = 16,
    target_modules=['q_proj' , 'v_proj', 'k_proj'] ,
    lora_dropout = 0.05,
    task_type = TaskType.CAUSAL_LM
)

model = get_peft_model(model ,lora_config)

In [ ]:
data = load_dataset('openai/gsm8k' ,'main',split='train[:200]')

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [6]:
def tokenize(batch):
    texts = [
        f"### Instruction:\n{inst}\n### Response:\n{out}"
        # for inst, out in zip(batch['question'], batch['answer'])
        for inst, out in zip(batch['instruction'], batch['response'])

    ]

    tokens = tokenizer(
        texts,
        padding = 'max_length',
        truncation = True,
        max_length = 256,
        return_tensors = 'pt'
    )

    tokens['labels'] = tokens['input_ids'].clone()

    return tokens

In [7]:
tokenized_data = data.map(tokenize,batched = True ,remove_columns=data.column_names)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [8]:
training_args = TrainingArguments(
    output_dir = './tinyllama-math-lora',
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4,
    learning_rate = 1e-3,
    num_train_epochs = 50,
    fp16 = True,
    save_total_limit = 3,
    report_to ='none',
    logging_steps = 20,
    save_strategy = 'epoch',
    remove_unused_columns = False,
    label_names = ['labels']
)

In [9]:
trainer = Trainer(
    model = model ,
    args = training_args,
    train_dataset = tokenized_data,
    processing_class = tokenizer
)

In [10]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Step,Training Loss
20,1.731900
40,0.810500
60,0.703700
80,0.609000
100,0.500800
120,0.416800
140,0.340200
160,0.250800
180,0.201800
200,0.150100


TrainOutput(global_step=650, training_loss=0.20423813939094543, metrics={'train_runtime': 1200.6327, 'train_samples_per_second': 8.329, 'train_steps_per_second': 0.541, 'total_flos': 1.593717030912e+16, 'train_loss': 0.20423813939094543, 'epoch': 50.0})

In [11]:
model.save_pretrained("./tinyllama-lora-tuned-adapter-math")
tokenizer.save_pretrained("./tinyllama-lora-tuned-adapter-math")

('./tinyllama-lora-tuned-adapter-math/tokenizer_config.json',
 './tinyllama-lora-tuned-adapter-math/special_tokens_map.json',
 './tinyllama-lora-tuned-adapter-math/chat_template.jinja',
 './tinyllama-lora-tuned-adapter-math/tokenizer.model',
 './tinyllama-lora-tuned-adapter-math/added_tokens.json',
 './tinyllama-lora-tuned-adapter-math/tokenizer.json')